In [1]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import torch

In [2]:
from huggingface_hub import notebook_login

In [15]:
notebook_login()

In [3]:
def load_csv_files(path):
    all_files = glob.glob(path + "/*.csv")
    li = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)
    return pd.concat(li, axis=0, ignore_index=True)

In [4]:
df = pd.read_csv('data/dataset.csv')

C:\Users\USER\AppData\Local\Temp\ipykernel_7108\344819524.py:1: DtypeWarning: Columns (3,4,7,8,9,10,12,19,20,26,27,28,29,31,32,33,34,35,36,37,41,44,45,46,49,51,55,56,57,58,59,61,65,72,73,74,75,77) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/dataset.csv')


In [10]:
df = df.dropna(subset=['price'])

In [14]:
df = df.reset_index(drop=True)

In [16]:
df = df.fillna(0)

In [5]:
df

,Unnamed: 0,name,price,type,form_factor,size,color,rpm,airflow,noise_level,...,capacity_w,capacity_va,memory,length,resolutions,connection,focus_type,os,fov,protocol
0,0,NZXT Hue,29.95,LED Controller,5.25,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0
1,1,Lian Li Uni Fan SL-Infinity 3-Pack,84.00,0,0,120.0,Black,0,"0,61.3","0,29",...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0
2,2,Corsair iCUE SP120 RGB ELITE 3-Pack,59.99,0,0,120.0,Black / White,"400,1500",0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0
3,3,Lian Li Uni Fan SL-Infinity 3-Pack,83.90,0,0,120.0,White,0,"0,61.3","0,29",...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0
4,4,Noctua NF-A12x25 PWM chromax.black.swap,34.95,0,0,120.0,Black,"450,2000",60.09,22.6,...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16496,16496,Hawking Technology HW17ACU,41.89,0,0,0.0,Black,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,Wi-Fi 5
16497,16497,Hawking Technology HW12ACU,52.99,0,0,0.0,Black,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,Wi-Fi 5
16498,16498,Sabrent USB-802N,25.66,0,0,0.0,Black,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,Wi-Fi 4
16499,16499,D-Link DWA-181-US,21.28,0,0,0.0,Black,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.0,Wi-Fi 5


In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [26]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

c:\Users\USER\anaconda3\envs\hi\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--TinyLlama--TinyLlama-1.1B-Chat-v1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [27]:
def prepare_data(df):
    data = []
    for _, row in df.iterrows():
        input_text = f"I need a {row['size']}mm fan with airflow around {row['airflow']} CFM and noise level around {row['noise_level']} dBA."
        output_text = f"Based on your requirements, I recommend the {row['name']} which costs ${row['price']}."
        data.append({"input": input_text, "output": output_text})
    return pd.DataFrame(data)



In [28]:
prepared_data = prepare_data(df)

In [29]:
class FanDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        input_encoding = self.tokenizer.encode_plus(
            item['input'],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        target_encoding = self.tokenizer.encode_plus(
            item['output'],
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encoding['input_ids'].flatten(),
            'attention_mask': input_encoding['attention_mask'].flatten(),
            'labels': target_encoding['input_ids'].flatten()
        }

In [30]:
train_data, val_data = train_test_split(prepared_data, test_size=0.2, random_state=42)
train_dataset = FanDataset(train_data, tokenizer)
val_dataset = FanDataset(val_data, tokenizer)

In [31]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

In [32]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [ ]:
trainer.train()

Step,Training Loss
